# SVM Classifier

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#loading in the datasets
df_training = pd.read_csv('../data/archive/train_pca.csv')
df_testing = pd.read_csv('../data/archive/test_pca.csv')

In [3]:
# get the training and test data

x_train = df_training.iloc[:, :-1]
y_train = df_training.iloc[:,-1]

x_test = df_testing.iloc[:, :-1]
y_test = df_testing.iloc[:,-1]

In [5]:
# Setting up the classifier

from sklearn import svm

clf = svm.SVC(decision_function_shape='ovr')
clf.fit(x_train, y_train)

y_predict = clf.predict(x_test)

In [7]:
# Scoring

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.6029860875466576
